# <span style="color:#0b486b">FIT3162 MCS13 Code</span>


---

## <span style="color:#0b486b">Imports</span>

Library imports required for this file. 
- `os` - OS related functions (getting/setting current directory or filepath(s))
- `zipfile` - to unzip files
- `PIL.Image` -  Image related functions
- `sklearn.model_selection.train_test_split` - A function to split up datasets
- `tensorflow` - General Deep Learning purposes
  - `keras`
    - `layers, models, activations` - for Deep Learning Model creation
- `matplotlib.pyplot` - for plotting graphs (if needed)
- `numpy` - for array types (needed for tensorflow functions)
- `BaseModel` - custom CNN Model class
- `ResNetModel` - custom CNN Model class, with ResNet blocks, Batch Normalisation and Dropout Layers
- `constants` - Constant values used throughout the file

In [1]:
""" Code Header
This file contains code thats trains and tests Models for Image Classification.
Various Model architectures are used and tried. 

In order to make use of the dataset files, ensure that the download.py file under the RealWorldOccludedFaces-main directory is run already.

Note: 
This jupyter file was obtained and altered from the Google Colab file from the MCS13 private Google Drive
The file was created on 08/03/2024 and contained code cells for: 
- Image loading
- Image resizing
- Creation of some simple models (obtained and altered from GeminiAI and Microsoft Copilot)
- Model training

@author MCS13
@version 1.1.0
@since 29/03/2024
@updated 30/03/2024
"""

# ============================================================================================================= #

# Imports 
import os
import zipfile
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, activations

import matplotlib.pyplot as plt
import numpy as np

from typing import List, Tuple
from numpy.typing import NDArray

from BaseModel import BaseModel
from ResNetModel import ResNetModel
from constants import *

plt.style.use('ggplot')
%matplotlib inline


---

## <span style="color:#0b486b">Loading Datasets</span>

***[RUN ONCE]*** Unzip and extract all files.

In [2]:
def unzip_file(filepath: str):
    """Unzips a zip file
    This code unzips a file and saves the content in the current directory.
    This means the content will be under the same directory as the original zip file.

    @since 1.0.0

    @prerequisite: The input filepath must end with ".zip"
    @raise ValueError: When the input filepath does not end with ".zip"

    @param filepath(str): The file path to unzip. The file path must lead to a zip file
    """
    # Pre-requisite check
    if filepath[-4:] != ".zip":
        raise ValueError(
            f"The input filepath must end with '.zip'. Expected filepath = '<FileName>.zip', got filepath = '{filepath}'."
        )

    # Unzip the file
    zip_obj = zipfile.ZipFile(file=filepath, mode="r")
    zip_obj.extractall("./")
    zip_obj.close()


"""RUN ONCE: Unzipping the dataset zip file """
# dataset_zip_filepath = f"{DATASET_NAME_MAIN}.zip"
# unzip_file(dataset_zip_filepath)

'RUN ONCE: Unzipping the dataset zip file '

Here, we start with the neutral faces dataset.

In [3]:
print(
    f'A few samples of people in the dataset: {os.listdir("RealWorldOccludedFaces-main/images/neutral")[:5]}'
)
print(
    f'Number of unique ids: {len(os.listdir("RealWorldOccludedFaces-main/images/neutral"))}'
)

A few samples of people in the dataset: ['adrien_brody', 'alain_delon', 'alexander_zverev', 'al_pacino', 'amber_heard']
Number of unique ids: 182


***[RUN ONCE]*** Resizes all images to a fixed shape of (64, 64)

In [4]:
# Main file paths
main_path = f"{DATASET_NAME_MAIN}/{NEUTRAL_DIR}"
main_dirs = os.listdir(main_path)
num_classes = len(main_dirs)

def resize_neutral_images(resized_shape: Tuple[int, int]):
    """Resizes all neutral images
    The function takes all images in RealWorldOccludedFaces-main/images/neutral
      and resizes them to 224x224 (determined by RESIZED_SHAPE constant)

    The resized shapes are then saved in RealWorldOccludedFaces-resized/images/neutral

    @param resized_shape(Tuple[int, int]): The shape to resize the shape to.
    """
    # Creates the directory paths
    resized_path = f"{DATASET_NAME_RESIZED}_{resized_shape[0]}/{NEUTRAL_DIR}"

    for path in [os.path.join(resized_path, f) for f in os.listdir(main_path)]:
        os.makedirs(f"{path}")
    
    # File paths of the neutral images
    paths = [
        os.path.join(resized_path, f)
        for f in main_dirs
    ]

    # Resizing and saving all images
    for name in os.listdir(main_path):
        for img_name in os.listdir(f"{main_path}/{name}"):
            Image.open(os.path.join(f"{main_path}/{name}/", img_name)).resize(
                resized_shape
            ).save(os.path.join(f"{resized_path}/{name}/", img_name))


# Creating the directory paths before resizing the images
# Needed before saving files
"""RUN ONCE: Resizing image dataset"""
# resize_neutral_images(RESIZED_SHAPES[0])

'RUN ONCE: Resizing image dataset'

Function to load the images into numpy arrays for model input.

In [5]:
def get_neutral_image_data(resized_shape: Tuple[int, int] = (64,64) ) -> (
    Tuple[NDArray[List[List[int]]], NDArray[NDArray[List[List[int]]]]]
):
    """Generates the dataset values (as ndarrays)
    Produces the dataset values (x-values) and
        the one-hot vector encoding of their labels (y-values)

    The values are obtained from the neutral images

    @rtype: Tuple[ndarray[List[List[int]]], ndarray[ndarray[List[List[uint8]]]]]
    @return: x-values, y-values
    """
    resized_path = f"{DATASET_NAME_RESIZED}_{resized_shape[0]}/{NEUTRAL_DIR}"

    # File paths of the neutral images
    paths = [
        os.path.join(resized_path, f)
        for f in main_dirs
    ]

    # Return the dataset values
    return np.array(
        [
            np.array(Image.open(img), "uint8")
            for path in paths
            for img in [os.path.join(path, f) for f in os.listdir(path)]
        ]
    ), np.array(
        [
            [int(bit == unique_id) for bit in range(len(os.listdir(main_path)))]
            for unique_id, path in enumerate(paths)
            for _ in [os.path.join(path, f) for f in os.listdir(path)]
        ]
    )

In [6]:
# Takes a while...
faces, ids = get_neutral_image_data(RESIZED_SHAPES[0])

In [7]:
print(f"Number of face images: {len(faces)}")  # size of neutral faces dataset
print(f"Number of ids (must match above): {len(ids)}")  # size of the array of face ids

Number of face images: 5927
Number of ids (must match above): 5927


Splitting the dataset into training, validation and testing.

In [8]:
# Splitting the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(
    faces, ids, test_size=0.2, random_state=42
)

# Splitting the training dataset into training and validation datasets
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)


--- 

## <span style="color:#0b486b">Modelling time!</span>

In [ ]:
model = BaseModel(
    name="Basic Model",
    input_width=RESIZED_SHAPES[0][0],
    input_height=RESIZED_SHAPES[0][1],
    depth=DEPTH,
    num_classes=num_classes,
    activation_func=RELU,
    optimiser=ADAM_OPT,
)

model.build_cnn()
model.summary()

In [ ]:
model.fit(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

Train the model

In [ ]:
evaluation_results = model.compute_accuracy(x_test, y_test)


---

## <span style="color:#0b486b">ResNet Upgrade!</span>

Now, we upgrade the model by implementing **ResNet blocks**, **Dropout layers** and **Batch Normalisation**

In [9]:
resnet_model = ResNetModel(
    name="Basic Model",
    input_width=RESIZED_SHAPES[0][0],
    input_height=RESIZED_SHAPES[0][1],
    depth=DEPTH,
    num_classes=num_classes,
    activation_func=RELU,
    optimiser=ADAM_OPT,
)

resnet_model.build_cnn()
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 96, 96, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 96, 96, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 96, 96, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 96, 96, 32)        0

In [10]:
resnet_model.fit(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

Epoch 1/20
134/134 [==============================] - 91s 670ms/step - loss: 5.1398 - accuracy: 0.0424
Epoch 2/20
134/134 [==============================] - 97s 725ms/step - loss: 3.6900 - accuracy: 0.2030
Epoch 3/20
134/134 [==============================] - 115s 857ms/step - loss: 2.6365 - accuracy: 0.4027
Epoch 4/20
134/134 [==============================] - 240s 2s/step - loss: 1.8318 - accuracy: 0.5881
Epoch 5/20
134/134 [==============================] - 116s 865ms/step - loss: 1.2875 - accuracy: 0.7211
Epoch 6/20
134/134 [==============================] - 96s 714ms/step - loss: 0.8952 - accuracy: 0.8279
Epoch 7/20
134/134 [==============================] - 95s 706ms/step - loss: 0.6256 - accuracy: 0.8915
Epoch 8/20
134/134 [==============================] - 94s 704ms/step - loss: 0.4419 - accuracy: 0.9372
Epoch 9/20
134/134 [==============================] - 94s 703ms/step - loss: 0.3013 - accuracy: 0.9674
Epoch 10/20
134/134 [==============================] - 95s 706ms/step - l

In [11]:
evaluation_results = resnet_model.compute_accuracy(x_test=x_test, y_test=y_test)

38/38 [==============================] - 7s 186ms/step - loss: 2.6314 - accuracy: 0.4983
loss: 2.631420135498047
accuracy: 0.49831366539001465


#### **Version Overview**

1.0.0
- Copied file from jupyter notebook in MCS13 private Google Drive
- Imported new BaseModel and constants files
- Did basic training using BaseModel 

1.1.0
- Imported ResNetModel